<a href="https://colab.research.google.com/github/PENROG21/PySpark/blob/main/%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговое задание  №6

Часть 1. Генерация информации
Мы использовали библиотеку Faker для генерации логов веб-сервера. Логи содержат следующую информацию:

IP-адрес клиента
Временная метка запроса
HTTP-метод (GET, POST, etc.)
URL запроса
Код ответа (200, 404, etc.)
Размер ответа в байтах
Сгенерировали 100,000 записей логов и сохранили их в CSV-файл.

In [ ]:
!pip install pyspark

In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.1 MB/s eta 0:00:00


In [ ]:
import csv
from faker import Faker
import random

fake = Faker()

num_records = 100000

http_methods = ['GET', 'POST', 'PUT', 'DELETE']
response_codes = [200, 301, 404, 500]

file_path = "web_server_logs.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ip', 'timestamp', 'method', 'url', 'response_code', 'response_size'])

    for _ in range(num_records):
        ip = fake.ipv4()
        timestamp = fake.date_time_this_year().isoformat()
        method = random.choice(http_methods)
        url = fake.uri_path()
        response_code = random.choice(response_codes)
        response_size = random.randint(100, 10000)

        writer.writerow([ip, timestamp, method, url, response_code, response_size])

print(f"Сгенерировано {num_records} записей и сохранено в {file_path}")

Сгенерировано 100000 записей и сохранено в web_server_logs.csv


In [ ]:
from pyspark.sql import SparkSession

In [27]:
from pyspark.sql.functions import count, col, sum, to_date

In [ ]:
spark = SparkSession.builder.appName("Nombaer_6").getOrCreate()

In [ ]:
df = spark.read.csv("/content/web_server_logs.csv", header=True, inferSchema=True)
df.printSchema()
df.show()

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

+---------------+--------------------+------+--------------------+-------------+-------------+
|             ip|           timestamp|method|                 url|response_code|response_size|
+---------------+--------------------+------+--------------------+-------------+-------------+
| 110.176.20.213|2025-01-20 23:08:...|   GET|search/posts/cate...|          404|         8388|
| 135.27.171.219|2025-03-02 09:11:...|   GET|category/wp-conte...|          500|         5211|
|  200.192.97.23|2025-02-16 00:03:...|   PUT|     app/blog/search|          301|         4609|
|   168.75.65.35|2025-01-18 14:01:...|   GET|          wp-content|          404|         7158|
|  177.224.51.45|2025-01-30 17:29:...|   PUT|    posts/wp-content|          404|      

In [ ]:
df = spark.read.csv("/content/web_server_logs.csv", header=True, inferSchema=True)
df.printSchema()
df.show()

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

+---------------+--------------------+------+--------------------+-------------+-------------+
|             ip|           timestamp|method|                 url|response_code|response_size|
+---------------+--------------------+------+--------------------+-------------+-------------+
| 110.176.20.213|2025-01-20 23:08:...|   GET|search/posts/cate...|          404|         8388|
| 135.27.171.219|2025-03-02 09:11:...|   GET|category/wp-conte...|          500|         5211|
|  200.192.97.23|2025-02-16 00:03:...|   PUT|     app/blog/search|          301|         4609|
|   168.75.65.35|2025-01-18 14:01:...|   GET|          wp-content|          404|         7158|
|  177.224.51.45|2025-01-30 17:29:...|   PUT|    posts/wp-content|          404|      

**Часть 2. Анализ информации**

1. Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP. Формат вывода, как на скрине ниже.

In [ ]:
df.groupBy("ip").agg(count("response_code").alias("request_count")).show(10)

+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
| 39.251.113.210|            1|
|  7.157.217.218|            1|
|   124.74.30.67|            1|
| 115.221.146.12|            1|
|  94.251.97.150|            1|
|   119.20.67.46|            1|
|  81.17.112.248|            1|
|  15.100.117.10|            1|
| 222.45.114.114|            1|
|152.127.149.165|            1|
+---------------+-------------+
only showing top 10 rows



2. Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.

In [13]:
df.groupBy("method").agg(count("method").alias("method_count")).show()

+------+------------+
|method|method_count|
+------+------------+
|  POST|       25005|
|DELETE|       24987|
|   PUT|       24970|
|   GET|       25038|
+------+------------+



3. Профильтруйте и посчитайте количество запросов с кодом ответа 404.

In [20]:
count_400 = df.filter(col("response_code") == 400).count()
print(f"Количество запросов с кодом 400: {count_400}")

Количество запросов с кодом 400: 0


4. Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

In [28]:
df.groupBy(to_date(col("timestamp"))).agg(sum("response_size").alias("total_response_size")).show()

+------------------+-------------------+
|to_date(timestamp)|total_response_size|
+------------------+-------------------+
|        2025-02-16|            7364991|
|        2025-02-01|            7128222|
|        2025-02-15|            7560309|
|        2025-02-05|            7258328|
|        2025-02-13|            7384224|
|        2025-02-06|            7247724|
|        2025-01-09|            7665605|
|        2025-02-12|            7582951|
|        2025-03-03|            7128705|
|        2025-02-22|            7440851|
|        2025-01-14|            7512980|
|        2025-03-08|            7590265|
|        2025-02-10|            7754612|
|        2025-02-28|            7670977|
|        2025-02-23|            7289590|
|        2025-01-18|            7281687|
|        2025-02-03|            7294701|
|        2025-01-28|            7327230|
|        2025-03-07|            7010965|
|        2025-01-17|            7435305|
+------------------+-------------------+
only showing top